In [1]:
# import the required packages for the execution of the program
import pandas as pd
import numpy as np
import tweepy
import json
from geopy.geocoders import Nominatim
import time

In [2]:
# save the twitter developer credentials for authentication
consumer_key = 'PekUwhQ8NaIB7kye74Zuwc5zN'
consumer_secret = 'ODZYsTbtG6JJrWZnQC8hARISSRcwN5uwiZdve3U9B0JbR7xXfg'
access_token = '832994996-2VqZdSdihKUdl73ONGtoPWfIIuV3l1xY0RG11cAr'
access_token_secret = 'crUrKJajF3f4puuVGgLNQxFZPShTikHuxg3Of0RRiSB0h'

In [3]:
# initialize authorization instance using credentials
authorization_instance = tweepy.OAuthHandler(consumer_key,consumer_secret)
authorization_instance.set_access_token(access_token, access_token_secret)
authorization_instance.secure = True

In [4]:
# initialize the twitter API object
twitter_api = tweepy.API(authorization_instance)
twitter_api

In [5]:
# define a function to fetch tweets based on location and query string
def get_tweets_by_location(query_string, address_string):
    geolocator = Nominatim()
    location = geolocator.geocode(address_string)
    print('Using cordinates for location:', location.address)
    latitude = str(location.latitude)
    longitude = str(location.longitude)
    radius='100mi'
    mygeocode = ','.join([latitude,longitude,radius])
    print('geocode:',mygeocode)
    custom_tweets = twitter_api.search(q = query_string, show_user = True, geocode = mygeocode, lang = 'en')
    print(len(custom_tweets),'tweets found for the given query.')    
    return custom_tweets

    
    

In [6]:
mytweets = get_tweets_by_location(query_string='RanveerSingh',
                      address_string='Mumbai, India')

Using cordinates for location: Mumbai, Greater Bombay, Mumbai Suburban, Maharashtra, India
geocode: 19.1334321,72.8882172,100mi
14 tweets found for the given query.


In [7]:
# iterate through the results returned from the function
for tweet in mytweets:
    print(tweet.text)

#AirportDiaries: @RanveerOfficial clicks a picture with Fans in his Funky Avataar
@RanveerSingh_FC @RanveerSinghFC… https://t.co/yHJS8Zgbkc
RT @LehrenNetworks: #UrvashiRautela #RanveerSingh #NehaBhasin #Lehren #30YearsOfLehren
Top 3 Bollywood celebs and their up to the mark airp…
RT @LehrenNetworks: #UrvashiRautela #RanveerSingh #NehaBhasin #Lehren #30YearsOfLehren
Top 3 Bollywood celebs and their up to the mark airp…
How amayyyyzzzinnng would it be to have these two beside you on a flight 🤩🤩 #AmeeshaPatel @ameesha_patel and… https://t.co/ugjz5eDMhb
RT @LehrenNetworks: #UrvashiRautela #RanveerSingh #NehaBhasin #Lehren #30YearsOfLehren
Top 3 Bollywood celebs and their up to the mark airp…
RT @bollybubble: #RanveerSingh’s back in the city and is not afraid to repeat his airport style! VIEW PICS

https://t.co/0jkH44edFH https:/…
RT @bollybubble: #RanveerSingh’s back in the city and is not afraid to repeat his airport style! VIEW PICS

https://t.co/0jkH44edFH https:/…
RT @viralphoto: Some 

In [8]:
# create a twitter stream listerner from the class provided by tweepy
class MyStreamListener(tweepy.StreamListener):
    def __init__(self,time_limit=60,count_limit = 10):
        self.start_time = time.time()
        self.count_limit = count_limit # restrict the number of tweets
        self.time_limit = time_limit  # restrict the time of streaming
        self.tweet_list = []
        self.count = 0
        self.output_file  = open('./logs.txt', 'w',encoding='utf-8') # open a csv file to save tweet logs
    
    def on_data(self, data):
        if (self.count_limit is not None and self.count >= self.count_limit): # check the tweet count
            self.output_file.close() # close the file
            return False
        elif  (time.time() - self.start_time) > self.time_limit: # check the time limit
            self.output_file.close()
            return False
        else:        
            self.count += 1
            json_data = json.loads(data)
            print(json_data)
            user_name = json_data['user']['screen_name']
#             if json_data['retweeted_status'] != None and json_data['retweeted_status']['extended_tweet']!= None:
                
#                 text = json_data['retweeted_status']['extended_tweet']['full_text']
#             else:
            text = json_data['text']
            time_stamp = json_data['created_at']

            self.tweet_list.append((user_name, text,time_stamp))

            self.output_file.write(''+'Tweet Count:'+str(self.count) +'\n'
                                   +'User Name:'+ user_name + '\n' 
                                   +'Tweet:'+text + '\n'
                                   +'Time Stamp:'+str(time_stamp) + '\n'
                                  +'===================================================< \n') # write the log to the csv file

            print(user_name,'==>',text,'<==',time_stamp,'\n') 
            return True
    
    def on_error(self, status):
        print(status)
    
    def on_timeout(self):
        print('Timeout...')
        return True
    
    def on_connect(self):
        print('Streaming is ON...')
    
    def on_disconnect(self):
        print('Streaming is disconnected...')
    
    def on_limit(self):
        print('Limit is crossed...')

In [9]:
# create a stream listener object with 'time_limit' seconds restriction
my_stream_listener = MyStreamListener(time_limit = 60, count_limit = 10)
twitter_streamer = tweepy.Stream(auth=twitter_api.auth,
                                listener = my_stream_listener,
                                tweet_mode='extended_tweet')

# specify a list of topics you are interested in 
topic_list = ['trump']
twitter_streamer.filter(track= topic_list,
                        async=False,
                        languages=['en'])


Streaming is ON...
{'created_at': 'Sat Apr 07 10:16:58 +0000 2018', 'id': 982562873594425345, 'id_str': '982562873594425345', 'text': '@SafetyPinDaily Oh no!\nwell, no worries, tRump will probably  help them out, they are his voters after all!', 'display_text_range': [16, 107], 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Lite</a>', 'truncated': False, 'in_reply_to_status_id': 982559369261969408, 'in_reply_to_status_id_str': '982559369261969408', 'in_reply_to_user_id': 449600949, 'in_reply_to_user_id_str': '449600949', 'in_reply_to_screen_name': 'SafetyPinDaily', 'user': {'id': 256923471, 'id_str': '256923471', 'name': '🇨🇦Isabelle🌈', 'screen_name': 'LeCorbeauBleu', 'location': 'Victoria BC, CANADA', 'url': None, 'description': 'HELPING THE RESISTANCE\nONE TWEET AT A TIME\n❤️Knowledge ❤️Sciences ❤️Arts ❤️Voyages ❤️ALL Life\n🌈LGBTQ is my community🇨🇦🌎🌍🌏🇫🇷', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1125, 'friends_count':

In [10]:
# print the tweet data saved in the file
f = open("./logs.txt","r", encoding="utf-8")
for line in f:
    print(line)

Tweet Count:1

User Name:LeCorbeauBleu

Tweet:@SafetyPinDaily Oh no!

well, no worries, tRump will probably  help them out, they are his voters after all!

Time Stamp:Sat Apr 07 10:16:58 +0000 2018

===================================================< 

Tweet Count:2

User Name:Dax_x98

Tweet:RT @Dax_x98: #FBR party



LIKE❤



RT♻️



FOLLOW ALL WHO RT🌊



@realDonaldTrump @VP @mike_pence #TheResistance #Resistance #ImpeachTrump #NotMyP…

Time Stamp:Sat Apr 07 10:16:58 +0000 2018

===================================================< 

Tweet Count:3

User Name:RSNightwatch

Tweet:The Memo: Trump’s risky gambit on trade roils markets https://t.co/h1Hn2S0eM6

Time Stamp:Sat Apr 07 10:16:59 +0000 2018

===================================================< 

Tweet Count:4

User Name:BeachCity55

Tweet:Will We Stop Trump Before It’s Too Late? https://t.co/8j1Cpq4ItU - We who?? Trump is president. YOU are not! Where… https://t.co/vlXdi1Zkbm

Time Stamp:Sat Apr 07 10:16:59 +0000 2018

========